# Deep Learning Face Recognition Model for UWA HSFD Database

This notebook implements a comprehensive face recognition system using deep learning techniques.
The model is designed for the UWA Hyperspectral Face Database (UWA HSFD).

## Table of Contents:
1. Environment Setup and Dependencies
2. Data Loading and Preprocessing
3. Exploratory Data Analysis
4. Data Augmentation
5. Model Architecture
6. Training Pipeline
7. Model Evaluation
8. Face Recognition and Authentication
9. Results Visualization

## 1. Environment Setup and Dependencies

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Scikit-learn utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Data Loading and Preprocessing

In [ ]:
# Configure data path
# Update this path to your local UWA HSFD database location
data_path = r"C:\Users\Anvitha\Face based Person Authentication\UWA HSFD V1.1 (1)\UWA HSFD V1.1\HyperSpec_Face_Session1"

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

In [ ]:
def load_hyperspectral_faces(data_path, img_size=(224, 224)):
    """
    Load hyperspectral face images from the UWA HSFD database.
    
    Args:
        data_path: Path to the hyperspectral face database
        img_size: Target image size for resizing
    
    Returns:
        images: Numpy array of processed images
        labels: Numpy array of corresponding labels
        label_names: Dictionary mapping label indices to person names
    """
    images = []
    labels = []
    label_names = {}
    
    if not os.path.exists(data_path):
        print(f"Warning: Data path '{data_path}' does not exist.")
        print("Creating sample synthetic data for demonstration...")
        return create_synthetic_data(img_size)
    
    # Walk through directory structure
    for person_idx, person_name in enumerate(sorted(os.listdir(data_path))):
        person_path = os.path.join(data_path, person_name)
        
        if os.path.isdir(person_path):
            label_names[person_idx] = person_name
            
            # Load all images for this person
            for img_file in os.listdir(person_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                    img_path = os.path.join(person_path, img_file)
                    
                    try:
                        # Load and preprocess image
                        img = cv2.imread(img_path)
                        
                        if img is not None:
                            # Convert BGR to RGB
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                            
                            # Resize image
                            img = cv2.resize(img, img_size)
                            
                            # Normalize pixel values to [0, 1]
                            img = img.astype('float32') / 255.0
                            
                            images.append(img)
                            labels.append(person_idx)
                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")
    
    if len(images) == 0:
        print("No images found. Creating sample synthetic data...")
        return create_synthetic_data(img_size)
    
    return np.array(images), np.array(labels), label_names

def create_synthetic_data(img_size, num_persons=10, images_per_person=20):
    """
    Create synthetic face data for demonstration purposes.
    """
    print(f"Creating synthetic dataset: {num_persons} persons, {images_per_person} images each")
    
    images = []
    labels = []
    label_names = {i: f"Person_{i+1}" for i in range(num_persons)}
    
    for person_idx in range(num_persons):
        for img_idx in range(images_per_person):
            # Create synthetic face-like images with person-specific patterns
            img = np.random.rand(img_size[0], img_size[1], 3).astype('float32')
            
            # Add person-specific pattern
            pattern = np.sin(np.linspace(0, 2*np.pi*person_idx, img_size[0]))
            img[:, :, 0] = img[:, :, 0] * 0.5 + pattern[:, np.newaxis] * 0.5
            
            images.append(img)
            labels.append(person_idx)
    
    return np.array(images), np.array(labels), label_names

In [ ]:
# Load the dataset
print("Loading hyperspectral face data...")
X, y, label_names = load_hyperspectral_faces(data_path, IMG_SIZE)

print(f"\nDataset loaded successfully!")
print(f"Number of images: {len(X)}")
print(f"Image shape: {X[0].shape}")
print(f"Number of unique persons: {len(np.unique(y))}")
print(f"Label names: {label_names}")

## 3. Exploratory Data Analysis

In [ ]:
# Visualize sample images from each class
def visualize_samples(X, y, label_names, samples_per_class=5):
    """
    Visualize sample images from each person in the dataset.
    """
    unique_labels = np.unique(y)
    n_classes = len(unique_labels)
    
    fig, axes = plt.subplots(n_classes, samples_per_class, figsize=(15, 3*n_classes))
    
    if n_classes == 1:
        axes = axes.reshape(1, -1)
    
    for idx, label in enumerate(unique_labels):
        # Get indices for this label
        label_indices = np.where(y == label)[0]
        
        # Sample random images
        sample_indices = np.random.choice(label_indices, 
                                         min(samples_per_class, len(label_indices)), 
                                         replace=False)
        
        for i, img_idx in enumerate(sample_indices):
            ax = axes[idx, i] if n_classes > 1 else axes[i]
            ax.imshow(X[img_idx])
            ax.axis('off')
            
            if i == 0:
                ax.set_title(f"{label_names[label]}", fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.suptitle('Sample Images from Dataset', fontsize=14, y=1.001)
    plt.show()

visualize_samples(X, y, label_names, samples_per_class=5)

In [ ]:
# Class distribution analysis
def plot_class_distribution(y, label_names):
    """
    Plot the distribution of images across different persons.
    """
    unique, counts = np.unique(y, return_counts=True)
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar([label_names[i] for i in unique], counts, color='skyblue', edgecolor='navy')
    plt.xlabel('Person ID', fontsize=12)
    plt.ylabel('Number of Images', fontsize=12)
    plt.title('Class Distribution in Dataset', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nClass Distribution Statistics:")
    print(f"Mean images per person: {np.mean(counts):.2f}")
    print(f"Std images per person: {np.std(counts):.2f}")
    print(f"Min images per person: {np.min(counts)}")
    print(f"Max images per person: {np.max(counts)}")

plot_class_distribution(y, label_names)

## 4. Data Preparation and Augmentation

In [ ]:
# Split data into train, validation, and test sets
# First split: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: 75% train, 25% val (of the temp set)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print("Data Split:")
print(f"Training set: {len(X_train)} images ({len(X_train)/len(X)*100:.1f}%)")
print(f"Validation set: {len(X_val)} images ({len(X_val)/len(X)*100:.1f}%)")
print(f"Test set: {len(X_test)} images ({len(X_test)/len(X)*100:.1f}%)")

# Convert labels to categorical
num_classes = len(np.unique(y))
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print(f"\nNumber of classes: {num_classes}")

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode='nearest'
)

# No augmentation for validation and test sets
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

print("Data augmentation configured successfully.")

## 5. Model Architecture

In [ ]:
def create_face_recognition_model(input_shape=(224, 224, 3), num_classes=10):
    """
    Create a deep learning model for face recognition.
    Uses transfer learning with ResNet50 as the base.
    
    Args:
        input_shape: Shape of input images
        num_classes: Number of persons to recognize
    
    Returns:
        model: Compiled Keras model
    """
    # Load pre-trained ResNet50 (without top layers)
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model layers initially
    base_model.trainable = False
    
    # Build custom top layers
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

def create_custom_cnn_model(input_shape=(224, 224, 3), num_classes=10):
    """
    Create a custom CNN model from scratch for face recognition.
    
    Args:
        input_shape: Shape of input images
        num_classes: Number of persons to recognize
    
    Returns:
        model: Compiled Keras model
    """
    model = models.Sequential([
        # First Conv Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second Conv Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third Conv Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fourth Conv Block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fully Connected Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

In [ ]:
# Create model - Choose one: Transfer Learning (faster, better) or Custom CNN
# Option 1: Transfer Learning with ResNet50 (Recommended)
print("Creating face recognition model with Transfer Learning (ResNet50)...")
model = create_face_recognition_model(input_shape=(224, 224, 3), num_classes=num_classes)

# Option 2: Custom CNN (uncomment to use)
# print("Creating custom CNN model...")
# model = create_custom_cnn_model(input_shape=(224, 224, 3), num_classes=num_classes)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

## 6. Training Pipeline

In [ ]:
# Define callbacks
callbacks = [
    # Save best model
    ModelCheckpoint(
        'best_face_recognition_model.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Training callbacks configured successfully.")

In [ ]:
# Train the model
print("Starting model training...")
print(f"Training for {EPOCHS} epochs with batch size {BATCH_SIZE}")

history = model.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=BATCH_SIZE),
    validation_data=(X_val, y_val_cat),
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\nTraining completed!")

In [ ]:
# Fine-tuning: Unfreeze some layers and retrain with lower learning rate
def fine_tune_model(model, X_train, y_train_cat, X_val, y_val_cat, epochs=20):
    """
    Fine-tune the model by unfreezing some base layers.
    """
    # Unfreeze the base model
    base_model = model.layers[0]
    if hasattr(base_model, 'trainable'):
        base_model.trainable = True
        
        # Freeze early layers, unfreeze later layers
        for layer in base_model.layers[:-30]:
            layer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Fine-tuning model...")
    
    history_fine = model.fit(
        train_datagen.flow(X_train, y_train_cat, batch_size=BATCH_SIZE),
        validation_data=(X_val, y_val_cat),
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    return history_fine

# Uncomment to perform fine-tuning
# history_fine = fine_tune_model(model, X_train, y_train_cat, X_val, y_val_cat, epochs=20)

## 7. Model Evaluation

In [ ]:
# Plot training history
def plot_training_history(history):
    """
    Plot training and validation accuracy and loss.
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    
    # Plot loss
    axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
# Evaluate on test set
print("Evaluating model on test set...")
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=1)

print(f"\nTest Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

In [ ]:
# Generate predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
print("\nClassification Report:")
print("=" * 70)
target_names = [label_names[i] for i in sorted(label_names.keys())]
print(classification_report(y_test, y_pred, target_names=target_names))

# Calculate additional metrics
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print(f"\nWeighted Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

In [ ]:
# Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, label_names):
    """
    Plot confusion matrix for model predictions.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=[label_names[i] for i in sorted(label_names.keys())],
                yticklabels=[label_names[i] for i in sorted(label_names.keys())],
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Calculate per-class accuracy
    class_accuracy = cm.diagonal() / cm.sum(axis=1)
    print("\nPer-Class Accuracy:")
    for i, acc in enumerate(class_accuracy):
        print(f"{label_names[i]}: {acc*100:.2f}%")

plot_confusion_matrix(y_test, y_pred, label_names)

## 8. Face Recognition and Authentication

In [ ]:
class FaceAuthenticator:
    """
    Face authentication system for verifying person identity.
    """
    def __init__(self, model, label_names, threshold=0.7):
        """
        Initialize the authenticator.
        
        Args:
            model: Trained face recognition model
            label_names: Dictionary mapping label indices to person names
            threshold: Confidence threshold for authentication (0-1)
        """
        self.model = model
        self.label_names = label_names
        self.threshold = threshold
    
    def preprocess_image(self, img_path, img_size=(224, 224)):
        """
        Preprocess an image for authentication.
        """
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        return img
    
    def authenticate(self, img_input, claimed_identity=None):
        """
        Authenticate a person from an image.
        
        Args:
            img_input: Image array or path to image
            claimed_identity: Expected person ID (for verification mode)
        
        Returns:
            Dictionary with authentication results
        """
        # Preprocess if path provided
        if isinstance(img_input, str):
            img = self.preprocess_image(img_input)
        else:
            img = np.expand_dims(img_input, axis=0) if len(img_input.shape) == 3 else img_input
        
        # Get predictions
        predictions = self.model.predict(img, verbose=0)
        predicted_class = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class]
        
        # Authentication result
        result = {
            'predicted_identity': self.label_names[predicted_class],
            'predicted_id': int(predicted_class),
            'confidence': float(confidence),
            'authenticated': confidence >= self.threshold,
            'all_probabilities': {self.label_names[i]: float(predictions[0][i]) 
                                 for i in range(len(predictions[0]))}
        }
        
        # Verification mode
        if claimed_identity is not None:
            result['claimed_identity'] = claimed_identity
            result['identity_match'] = (predicted_class == claimed_identity)
            result['verified'] = (result['identity_match'] and result['authenticated'])
        
        return result
    
    def batch_authenticate(self, images):
        """
        Authenticate multiple images.
        """
        results = []
        for img in images:
            results.append(self.authenticate(img))
        return results

# Create authenticator instance
authenticator = FaceAuthenticator(model, label_names, threshold=0.7)
print("Face Authenticator created successfully!")
print(f"Authentication threshold: {authenticator.threshold}")

In [ ]:
# Test authentication on random test samples
def test_authentication(authenticator, X_test, y_test, label_names, num_samples=10):
    """
    Test authentication on random samples from test set.
    """
    sample_indices = np.random.choice(len(X_test), num_samples, replace=False)
    
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    axes = axes.ravel()
    
    for idx, sample_idx in enumerate(sample_indices):
        img = X_test[sample_idx]
        true_label = y_test[sample_idx]
        
        # Authenticate
        result = authenticator.authenticate(img, claimed_identity=true_label)
        
        # Display
        axes[idx].imshow(img)
        axes[idx].axis('off')
        
        title_color = 'green' if result['verified'] else 'red'
        title = f"True: {label_names[true_label]}\n"
        title += f"Pred: {result['predicted_identity']}\n"
        title += f"Conf: {result['confidence']:.2%}\n"
        title += f"Status: {'✓ VERIFIED' if result['verified'] else '✗ REJECTED'}"
        
        axes[idx].set_title(title, fontsize=9, color=title_color, fontweight='bold')
    
    plt.tight_layout()
    plt.suptitle('Face Authentication Test Results', fontsize=14, fontweight='bold', y=1.001)
    plt.show()

test_authentication(authenticator, X_test, y_test, label_names, num_samples=10)

In [ ]:
# Calculate authentication metrics
def evaluate_authentication_system(authenticator, X_test, y_test):
    """
    Evaluate authentication system performance.
    """
    results = authenticator.batch_authenticate(X_test)
    
    # Calculate metrics
    total = len(results)
    authenticated = sum(1 for r in results if r['authenticated'])
    correct_predictions = sum(1 for i, r in enumerate(results) if r['predicted_id'] == y_test[i])
    
    # For verification scenario
    verified_results = [authenticator.authenticate(X_test[i], claimed_identity=y_test[i]) 
                       for i in range(len(X_test))]
    verified = sum(1 for r in verified_results if r['verified'])
    
    print("Authentication System Performance:")
    print("=" * 50)
    print(f"Total test samples: {total}")
    print(f"Authenticated (confidence ≥ threshold): {authenticated} ({authenticated/total*100:.2f}%)")
    print(f"Correct predictions: {correct_predictions} ({correct_predictions/total*100:.2f}%)")
    print(f"Verified (correct + confident): {verified} ({verified/total*100:.2f}%)")
    print(f"\nFalse Rejection Rate (FRR): {(total-verified)/total*100:.2f}%")
    
    # Confidence distribution
    confidences = [r['confidence'] for r in results]
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(confidences, bins=20, color='skyblue', edgecolor='navy')
    plt.axvline(authenticator.threshold, color='red', linestyle='--', linewidth=2, label='Threshold')
    plt.xlabel('Confidence Score', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title('Confidence Score Distribution', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    metrics = ['Authenticated', 'Correct\nPredictions', 'Verified']
    values = [authenticated/total*100, correct_predictions/total*100, verified/total*100]
    bars = plt.bar(metrics, values, color=['skyblue', 'lightgreen', 'coral'], edgecolor='navy')
    plt.ylabel('Percentage (%)', fontsize=12)
    plt.title('Authentication Metrics', fontsize=14, fontweight='bold')
    plt.ylim(0, 105)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

evaluate_authentication_system(authenticator, X_test, y_test)

## 9. Model Persistence and Deployment

In [ ]:
# Save the complete model
model.save('face_recognition_model_complete.h5')
print("Complete model saved as 'face_recognition_model_complete.h5'")

# Save model in TensorFlow SavedModel format (recommended for deployment)
model.save('face_recognition_model_savedmodel', save_format='tf')
print("Model saved in TensorFlow SavedModel format")

# Save label names
import json
with open('label_names.json', 'w') as f:
    json.dump(label_names, f, indent=2)
print("Label names saved as 'label_names.json'")

In [ ]:
# Load model for inference (example)
def load_model_for_inference(model_path='face_recognition_model_complete.h5', 
                            label_path='label_names.json'):
    """
    Load a saved model for inference.
    """
    loaded_model = keras.models.load_model(model_path)
    
    with open(label_path, 'r') as f:
        loaded_labels = json.load(f)
        # Convert string keys back to integers
        loaded_labels = {int(k): v for k, v in loaded_labels.items()}
    
    return loaded_model, loaded_labels

# Example: Load and test
# loaded_model, loaded_labels = load_model_for_inference()
# print("Model loaded successfully for inference!")

## 10. Summary and Next Steps

### Model Performance Summary:
This notebook has successfully implemented a deep learning-based face recognition system with the following components:

1. **Data Pipeline**: Robust data loading and preprocessing for hyperspectral face images
2. **Model Architecture**: Transfer learning with ResNet50 or custom CNN
3. **Training**: Complete training pipeline with data augmentation and callbacks
4. **Evaluation**: Comprehensive metrics including accuracy, precision, recall, and F1-score
5. **Authentication**: Face authentication system with confidence thresholding

### Key Features:
- Data augmentation for improved generalization
- Transfer learning for faster convergence
- Authentication with confidence thresholding
- Comprehensive evaluation metrics
- Model persistence for deployment

### Next Steps for Production:
1. **Collect More Data**: Increase dataset size for better generalization
2. **Face Detection**: Add face detection preprocessing (e.g., MTCNN, Haar Cascades)
3. **Face Alignment**: Implement face alignment for better recognition
4. **Liveness Detection**: Add anti-spoofing measures
5. **API Development**: Create REST API for model deployment
6. **Real-time Processing**: Optimize for real-time video stream processing
7. **Database Integration**: Connect to authentication database
8. **Logging and Monitoring**: Add comprehensive logging system

### Usage Instructions:
```python
# To use the saved model:
model, labels = load_model_for_inference()
authenticator = FaceAuthenticator(model, labels, threshold=0.7)

# Authenticate a face:
result = authenticator.authenticate('path/to/face/image.jpg')
print(f"Identity: {result['predicted_identity']}")
print(f"Confidence: {result['confidence']:.2%}")
print(f"Authenticated: {result['authenticated']}")
```